In [1]:
import sys
from os.path import dirname
sys.path.append(dirname(__name__)+'..')

import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.loader import DataLoader
import torch.nn.functional as F

# from models.layer import MaxPooling, MaxPoolingX
import torch_geometric


from torch_geometric.utils import to_dense_batch, to_dense_adj, degree, add_remaining_self_loops

import logging

from sklearn.metrics import roc_auc_score

In [2]:
dataset = torch_geometric.datasets.TUDataset('../data/', 'IMDB-BINARY')

idx = torch.randperm(len(dataset))
cut_off = int(len(dataset) * 0.8)
train_dataset = dataset[idx[:cut_off]]
test_dataset = dataset[idx[cut_off:]]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# dataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
print(f'training size {len(train_dataset)}, testing set {len(test_dataset)}, Number of graphs per batch: {next(iter(train_loader)).y.shape[0]}')


cuda
training size 800, testing set 200, Number of graphs per batch: 16


In [3]:
feature_dim = 0
for g in dataset:
    deg = degree(g.edge_index[0], g.num_nodes)
    feature_dim = max(feature_dim, deg.max().item())
print(f'feature_dim: {feature_dim}')
FEAT_DIM = int(feature_dim)+1 if feature_dim < 400 else 400

def gen_node_feature(data):
    deg = degree(data.edge_index[0], data.num_nodes).long()
    deg[deg >= FEAT_DIM] = FEAT_DIM - 1
    feat = F.one_hot(deg, num_classes=FEAT_DIM).float()
    return feat

feature_dim: 135.0


In [ ]:
def calculate_metrics(true_labels, predicted_labels):
    ra = roc_auc_score(true_labels, predicted_labels)
    return ra

def evaluate(model, train_loader, device):
    model.eval()

    trueLabel = []
    predLabel = []
    with torch.no_grad():
        # for data in tqdm.tqdm(train_loader):
        for data in train_loader:
            data = data.to(device)
            data.edge_index = add_remaining_self_loops(data.edge_index)[0]
            hs, mask = to_dense_batch(data.x, data.batch)
            hs = [hs[i][mask[i]] for i in range(len(hs))]
            gs = to_dense_adj(data.edge_index, data.batch)
            gs = [gs[i][:len(hs[i]), :len(hs[i])] for i in range(len(gs))]
            _, o_gs = model(gs, hs)

            for og, g in zip(o_gs, gs):
                # og = (torch.sign(og-0.5)+1)/2
                trueLabel += g.int().cpu().numpy().flatten().tolist()
                predLabel += og.cpu().numpy().flatten().tolist()
    trueLabel = np.array(trueLabel)
    predLabel = np.array(predLabel)
    return calculate_metrics(trueLabel, predLabel)

class ARG():
    pass

# variational

In [4]:
from GraphUNET.ops import GCN, norm_g
from GraphCroc.UNET import Encoder, Decoder

In [5]:
class Unet(nn.Module):
	'''
	two-way network
	'''
	def __init__(self, in_dim=None, args=None, s_gcn_state=None, encoder_state=None, s_ln_state=None) -> None:
		super(Unet, self).__init__()
		self.act = getattr(nn, args.act)()
		self.mask_ratio = args.mask_ratio

		self.s_gcn = GCN(in_dim, args.dim, self.act, args.drop_p)
		self.s_ln = nn.LayerNorm(args.dim)
		if s_gcn_state:
			self.s_gcn.load_state_dict(s_gcn_state)
			for param in self.s_gcn.parameters(): # freeze the grad of source gcn
				param.requires_grad = False
		if s_ln_state:
			self.s_ln.load_state_dict(s_ln_state)
			for param in self.s_ln.parameters(): # freeze the grad of source gcn
				param.requires_grad = False

		self.g_enc = Encoder(args.ks, args.dim, self.act, args.drop_p)
		if encoder_state:
			self.g_enc.load_state_dict(encoder_state)
			for param in self.g_enc.parameters(): # freeze the grad of encoder
				param.requires_grad = False

		self.bot_gcn = GCN(args.dim, args.dim, self.act, args.drop_p)
		self.bot_ln = nn.LayerNorm(args.dim)
		self.g_dec_mean1 = Decoder(args.ks, args.dim, self.act, args.drop_p)
		self.g_dec_logstd1 = Decoder(args.ks, args.dim, self.act, args.drop_p)
		self.g_dec_mean2 = Decoder(args.ks, args.dim, self.act, args.drop_p)
		self.g_dec_logstd2 = Decoder(args.ks, args.dim, self.act, args.drop_p)
	
	def forward(self, gs, hs):
		o_gs = self.embed(gs, hs)
		return self.customBCE(o_gs, gs), o_gs
	
	def embed(self, gs, hs):
		o_gs = []
		for g, h in zip(gs, hs):
			og = self.embed_one(g, h)
			o_gs.append(og)
		return o_gs

	def embed_one(self, g, h):
		g = norm_g(g)
		h = self.s_gcn(g, h)
		h = self.s_ln(h)
		ori_h = h
		g, h, adj_ms, down_outs, indices_list = self.g_enc(g, h)

		g = norm_g(g)
		h = self.bot_gcn(g, h)
		h = self.bot_ln(h)
		mean1 = self.g_dec_mean1(h, ori_h, down_outs, adj_ms, indices_list)
		std1 = self.g_dec_logstd1(h, ori_h, down_outs, adj_ms, indices_list)
		mean2 = self.g_dec_mean2(h, ori_h, down_outs, adj_ms, indices_list)
		std2 = self.g_dec_logstd2(h, ori_h, down_outs, adj_ms, indices_list)

		h1 = torch.randn(mean1.size()).to(device)
		h1 = h1*torch.exp(std1) + mean1
		h2 = torch.randn(mean2.size()).to(device)
		h2 = h2*torch.exp(std2) + mean2

		h = (h1 @ h2.T)
		return torch.sigmoid((h+h.T)/2)

	def customBCE(self, o_gs, gs):
		loss = 0
		cnts = 0
		for og, g in zip(o_gs, gs):
			tn = g.numel()
			zeros = tn - g.sum()
			ones = g.sum()
			one_weight = tn / 2 / ones
			zero_weight = tn / 2 / zeros
			weights = torch.where(g == 0, zero_weight, one_weight)
			loss += F.binary_cross_entropy(og, g, weight=weights)
			cnts += 1
		loss /= cnts
		return loss

In [6]:
def train(model, train_loader: DataLoader, optimizer, scheduler, device, epoch):
    global writer
    model.train()

    total_loss = 0
    for data in train_loader:
        data.x = gen_node_feature(data)
        data = data.to(device)
        data.edge_index = add_remaining_self_loops(data.edge_index)[0]
        optimizer.zero_grad()
        label = data.y
        hs, mask = to_dense_batch(data.x, data.batch)
        hs = [hs[i][mask[i]] for i in range(len(hs))]
        gs = to_dense_adj(data.edge_index, data.batch)
        gs = [gs[i][:len(hs[i]), :len(hs[i])] for i in range(len(gs))]
        loss, _ = model(gs, hs)

        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    scheduler.step()

def calculate_metrics(true_labels, predicted_labels):
    ra = roc_auc_score(true_labels, predicted_labels)
    return ra

def evaluate(model, train_loader, device):
    model.eval()

    trueLabel = []
    predLabel = []
    with torch.no_grad():
        for data in train_loader:
            data.x = gen_node_feature(data)
            data = data.to(device)
            data.edge_index = add_remaining_self_loops(data.edge_index)[0]
            hs, mask = to_dense_batch(data.x, data.batch)
            hs = [hs[i][mask[i]] for i in range(len(hs))]
            gs = to_dense_adj(data.edge_index, data.batch)
            gs = [gs[i][:len(hs[i]), :len(hs[i])] for i in range(len(gs))]
            _, o_gs = model(gs, hs)

            for og, g in zip(o_gs, gs):
                trueLabel += g.int().cpu().numpy().flatten().tolist()
                predLabel += og.cpu().numpy().flatten().tolist()
    trueLabel = np.array(trueLabel)
    predLabel = np.array(predLabel)
    return (trueLabel==predLabel).sum()/trueLabel.size, calculate_metrics(trueLabel, predLabel)

In [7]:
class ARG():
    pass

args = ARG()
args.input_dim = FEAT_DIM
args.act = 'GELU'
args.dim = 128
args.drop_p = 0
args.mask_ratio = 0.5
args.ks = [0.9, 0.8, 0.7, 0.6, 0.5] + [0]
num_epoch = 200

model = Unet(in_dim= args.input_dim, args=args).to(device)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epoch)

In [8]:
logging.basicConfig(filename='resLog/result_variational.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

train_acc = []
test_acc = []
train_acc.append(evaluate(model, train_loader, device))
test_acc.append(evaluate(model, test_loader, device))
logging.info(f'Epoch [0/{num_epoch}], acc_train: {train_acc[-1][0]:.6f}, acc_test: {test_acc[-1][0]:.6f}, roc_train: {train_acc[-1][-1]:.6f}, roc_test: {test_acc[-1][-1]:.6f}')
for epoch in range(num_epoch):
    train(model, train_loader, optimizer, scheduler, device, epoch)
    train_acc.append(evaluate(model, train_loader, device))
    test_acc.append(evaluate(model, test_loader, device))
    logging.info(f'Epoch [{epoch+1}/{num_epoch}], acc_train: {train_acc[-1][0]:.6f}, acc_test: {test_acc[-1][0]:.6f}, roc_train: {train_acc[-1][-1]:.6f}, roc_test: {test_acc[-1][-1]:.6f}')

# edge masking

In [9]:
from GraphCroc.UNET import Unet

In [10]:
class newUnet(Unet):
    def forward(self, gs, hs, ori_gs):
        o_gs = self.embed(gs, hs)
        return self.customBCE(o_gs, ori_gs), o_gs

In [11]:
def apply_undirected_mask(gs, mask_ratio):
    new_gs = []
    for g in gs:
        mask = (torch.rand_like(g) > mask_ratio).float()
        mask = torch.triu(mask, diagonal=1).to(device)
        mask = mask + mask.T + torch.eye(mask.size(0)).to(device)
        g = g * mask
        new_gs.append(g)
    return new_gs

def train(model, train_loader: DataLoader, optimizer, scheduler, device, mask_ratio, epoch):
    global writer
    model.train()

    total_loss = 0
    for data in train_loader:
        data.x = gen_node_feature(data)
        data = data.to(device)
        data.edge_index = add_remaining_self_loops(data.edge_index)[0]
        optimizer.zero_grad()
        label = data.y
        hs, mask = to_dense_batch(data.x, data.batch)
        hs = [hs[i][mask[i]] for i in range(len(hs))]
        gs = to_dense_adj(data.edge_index, data.batch)
        gs = [gs[i][:len(hs[i]), :len(hs[i])] for i in range(len(gs))]
        new_gs = apply_undirected_mask(gs, mask_ratio)
        loss, _ = model(new_gs, hs, new_gs)

        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    scheduler.step()

def calculate_metrics(true_labels, predicted_labels):
    ra = roc_auc_score(true_labels, predicted_labels)
    return ra

def evaluate(model, train_loader, device):
    model.eval()

    trueLabel = []
    predLabel = []
    with torch.no_grad():
        for data in train_loader:
            data.x = gen_node_feature(data)
            data = data.to(device)
            data.edge_index = add_remaining_self_loops(data.edge_index)[0]
            hs, mask = to_dense_batch(data.x, data.batch)
            hs = [hs[i][mask[i]] for i in range(len(hs))]
            gs = to_dense_adj(data.edge_index, data.batch)
            gs = [gs[i][:len(hs[i]), :len(hs[i])] for i in range(len(gs))]
            _, o_gs = model(gs, hs, gs)

            for og, g in zip(o_gs, gs):
                trueLabel += g.int().cpu().numpy().flatten().tolist()
                predLabel += og.cpu().numpy().flatten().tolist()
    trueLabel = np.array(trueLabel)
    predLabel = np.array(predLabel)
    return (trueLabel==predLabel).sum()/trueLabel.size, calculate_metrics(trueLabel, predLabel)

In [12]:
class ARG():
    pass

args = ARG()
args.input_dim = FEAT_DIM
args.act = 'GELU'
args.dim = 128
args.drop_p = 0
args.mask_ratio = 0.5
args.ks = [0.9, 0.8, 0.7, 0.6, 0.5] + [0]


num_epoch = 200

model = newUnet(in_dim=args.input_dim, args=args).to(device)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epoch)

In [13]:
logging.basicConfig(filename='resLog/result_masking.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

train_acc = []
test_acc = []
train_acc.append(evaluate(model, train_loader, device))
test_acc.append(evaluate(model, test_loader, device))
logging.info(f'Epoch [0/{num_epoch}], acc_train: {train_acc[-1][0]:.6f}, acc_test: {test_acc[-1][0]:.6f}, roc_train: {train_acc[-1][-1]:.6f}, roc_test: {test_acc[-1][-1]:.6f}')
for epoch in range(num_epoch):
    train(model, train_loader, optimizer, scheduler, device, 0.5, epoch)
    train_acc.append(evaluate(model, train_loader, device))
    test_acc.append(evaluate(model, test_loader, device))
    logging.info(f'Epoch [{epoch+1}/{num_epoch}], acc_train: {train_acc[-1][0]:.6f}, acc_test: {test_acc[-1][0]:.6f}, roc_train: {train_acc[-1][-1]:.6f}, roc_test: {test_acc[-1][-1]:.6f}')

# L2-norm

In [14]:
from GraphUNET.ops import norm_g
from GraphCroc.UNET import Unet

In [15]:
class newUnet(Unet):
    def embed_one(self, g, h):
        g = norm_g(g)
        h = self.s_gcn(g, h)
        h = self.s_ln(h)
        ori_h = h
        g, h, adj_ms, down_outs, indices_list = self.g_enc(g, h)

        g = norm_g(g)
        h = self.bot_gcn(g, h)
        h = self.bot_ln(h)
        h1 = self.g_dec1(h, ori_h, down_outs, adj_ms, indices_list)
        h2 = self.g_dec2(h, ori_h, down_outs, adj_ms, indices_list)
        h = torch.cdist(h1, h2, p=2)
        h = 10 * (1-h)

        return torch.sigmoid(h)

In [16]:
def train(model, train_loader: DataLoader, optimizer, scheduler, device, mask_ratio, epoch):
    global writer
    model.train()

    total_loss = 0
    for data in train_loader:
        data.x = gen_node_feature(data)
        data = data.to(device)
        data.edge_index = add_remaining_self_loops(data.edge_index)[0]
        optimizer.zero_grad()
        label = data.y
        # data.x = degree(data.edge_index[1], num_nodes=data.num_nodes).unsqueeze_(-1)
        hs, mask = to_dense_batch(data.x, data.batch)
        hs = [hs[i][mask[i]] for i in range(len(hs))]
        gs = to_dense_adj(data.edge_index, data.batch)
        gs = [gs[i][:len(hs[i]), :len(hs[i])] for i in range(len(gs))]
        loss, _ = model(gs, hs)

        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    scheduler.step()

def calculate_metrics(true_labels, predicted_labels):
    ra = roc_auc_score(true_labels, predicted_labels)
    return ra

def evaluate(model, train_loader, device):
    model.eval()

    trueLabel = []
    predLabel = []
    with torch.no_grad():
        # for data in tqdm.tqdm(train_loader):
        for data in train_loader:
            data.x = gen_node_feature(data)
            data = data.to(device)
            data.edge_index = add_remaining_self_loops(data.edge_index)[0]
            hs, mask = to_dense_batch(data.x, data.batch)
            hs = [hs[i][mask[i]] for i in range(len(hs))]
            gs = to_dense_adj(data.edge_index, data.batch)
            gs = [gs[i][:len(hs[i]), :len(hs[i])] for i in range(len(gs))]
            _, o_gs = model(gs, hs)

            for og, g in zip(o_gs, gs):
                # og = (torch.sign(og-0.5)+1)/2
                trueLabel += g.int().cpu().numpy().flatten().tolist()
                predLabel += og.cpu().numpy().flatten().tolist()
    trueLabel = np.array(trueLabel)
    predLabel = np.array(predLabel)
    return (trueLabel==predLabel).sum()/trueLabel.size, calculate_metrics(trueLabel, predLabel)

In [17]:
class ARG():
    pass

args = ARG()
args.input_dim = FEAT_DIM
args.act = 'GELU'
args.dim = 128
args.drop_p = 0
args.mask_ratio = 0.5
args.ks = [0.9, 0.8, 0.7, 0.6, 0.5] + [0]


num_epoch = 200

model = newUnet(in_dim=args.input_dim, args=args).to(device)
optimizer = torch.optim.AdamW(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epoch)

In [18]:
logging.basicConfig(filename='resLog/result_l2norm.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

train_acc = []
test_acc = []
train_acc.append(evaluate(model, train_loader, device))
test_acc.append(evaluate(model, test_loader, device))
logging.info(f'Epoch [0/{num_epoch}], acc_train: {train_acc[-1][0]:.6f}, acc_test: {test_acc[-1][0]:.6f}, roc_train: {train_acc[-1][-1]:.6f}, roc_test: {test_acc[-1][-1]:.6f}')
for epoch in range(num_epoch):
    train(model, train_loader, optimizer, scheduler, device, 0.5, epoch)
    train_acc.append(evaluate(model, train_loader, device))
    test_acc.append(evaluate(model, test_loader, device))
    logging.info(f'Epoch [{epoch+1}/{num_epoch}], acc_train: {train_acc[-1][0]:.6f}, acc_test: {test_acc[-1][0]:.6f}, roc_train: {train_acc[-1][-1]:.6f}, roc_test: {test_acc[-1][-1]:.6f}')